# YouTube scraping

Importiamo le librerie necessarrie:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

Leggiamo i dati presi da Spotify:

In [ ]:
df = pd.read_csv("spotify.csv")

Creiamo delle liste vuote

In [ ]:
URL=[]
traccia=[]
artista=[]

Il seguente codice utilizza la libreria Selenium per automatizzare l'interazione con il sito web di YouTube. 
Il codice crea un oggetto ``options`` di tipo Options e imposta la strategia di caricamento della pagina su "normal". Quindi, viene creato un oggetto ``driver`` di tipo ``webdriver.Chrome`` utilizzando le opzioni specificate.

Successivamente, viene caricata la pagina iniziale di YouTube e cliccato sul pulsante per rifiutare i cookie.

Nel ciclo for, per ogni riga del DataFrame ``df`` che contiene i valori Artist e Track, viene effettuata una ricerca su YouTube utilizzando la stringa composta dall'artista e dalla traccia seguiti dalla parola chiave "official video". Il primo risultato viene cliccato e l'URL viene recuperato. L'URL, il nome della traccia e il nome dell'artista vengono quindi salvati rispettivamente nella lista ``url``, ``traccia`` e ``artista``.

In [ ]:
#open Chrome
options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)

#load youtube
driver.get("https://www.youtube.com")

wait = WebDriverWait(driver, 10)
#reject cookies   
reject = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="content"]/div[2]/div[6]/div[1]/ytd-button-renderer[1]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')))
reject.click()
#iterate on df
for artist,track in zip(df['Artist'], df['Track']):
    #search for the video
    query = artist + " " + track + " official video"
    driver.get(f"https://www.youtube.com/results?search_query={query}")
    #click on the first result and get the URL
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="thumbnail"]/yt-image/img')))
        element.click()
    except:
        element1 = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="video-title"]/yt-formatted-string')))
        element1.click()
        
    url.append(driver.current_url)
    traccia.append(track)
    artista.append(artist)

# YouTube Api

Una volta ottenuti gli URL dei video di YouTube raccogliamo le informazioni che ci interessano (visualizzazioni, numero commenti, likes, titolo del video, nome del canale), utilizzando l'API di YouTube. 

Importiamo le librerie:

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
import re

Definiamo una funzione che estrae l'ID dei video dall'URL. Ci servirà come parametro da inserire nella richiesta 

In [ ]:
def extract_video_id(url):
    # Extract the video ID from the URL using a regular expression
    match = re.search(r"v=([a-zA-Z0-9-_]{11})", url)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid YouTube URL")

Creiamo liste vuote: 

In [ ]:
views=[]
titles=[]
channels=[]
likes=[]
comments=[]
descriptions=[]
licensed=[]

Fornire chiave API

In [ ]:
API_KEY="Your_Key"

Il seguente codice ci consente di ottenere le informazioni di cui abbiamo bisogno utilizzando gli URL che abbiamo raschiato.

In [ ]:
youtube = build("youtube", "v3", developerKey=API_KEY)
for url in URL[0:]:
    #check for valid url
    try:
        video_id=extract_video_id(url)
    except:
        #if the video is a youtube short the API won't recognize its id so, we won't consider those.
        channels.append(None)
        titles.append(None)
        views.append(None)
        comments.append(None)
        likes.append(None)
        descriptions.append(None)
        licensed.append(None)
        print(f'short on {url}')
    else:

        request = youtube.videos().list(part="statistics", id=video_id)
        request1 =youtube.videos().list(part="snippet", id=video_id)
        request2 = youtube.videos().list(part="contentDetails", id=video_id)
        
        response = request.execute()
        response1 =request1.execute()
        response2 = request2.execute()
        
        #Numero visualizzazioni
        try: 
            view=int(response["items"][0]["statistics"]["viewCount"])
            views.append(view)
        except:
            views.appemd(None)
            print(f'views error on {url}')
        #Numero like
        try:
            like=int(response["items"][0]["statistics"]["likeCount"])
            likes.append(like)
        except:
            likes.append(None)
            print(f'like error on {url}')
        #Numero Commenti
        try:
            comment=int(response["items"][0]["statistics"]["commentCount"])
            comments.append(comment)
        except:
            comments.append(None)
            print(f'comment error on {url}')
        #Titolo Video
        try:
            title=response1["items"][0]["snippet"]["title"]
            titles.append(title)
        except:
            titles.append(None)
            print(f'title error on {url}')
        #Nome Canale
        try:
            channel=response1["items"][0]["snippet"]["channelTitle"]
            channels.append(channel)
        except:
            channels.append(None)
            print(f'channel error on {url}')
        #Descrizione
        try:
            desc=response1["items"][0]["snippet"]["description"]
            descriptions.append(desc)
        except:
            descriptions.append(None)
            print(f'descriptions error on {url}')
        #Licensed
        try:
            lic=response2["items"][0]["contentDetails"]["licensedContent"]
            licensed.append(lic)
        except:
            licensed.append(None)
            print(f'licensed error on {url}')
        

Memorizziamo il risultato in un dataframe ```yt``` e salviamo il file csv

In [ ]:
yt = pd.DataFrame({"Artist":artista, "Track": traccia, "Url": URL, "Channel": channels, "Title":titles, "Views":views,
                   "Likes":likes, "Comments":comments, "Description":descriptions, "Licensed": licensed})

In [ ]:
yt.to_csv('Youtube_url_views_final.csv')